# Darcy equation

In this tutorial we present how to solve a Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknowns are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability and $g$ a vector source term, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {g}\\
\nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \qquad p = 1 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we will use a Raviart-Thomas approximation for ${q}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 1d and 3d. PyGeoN works with mixed-dimensional grids, so we need to convert the grid.

In [2]:
N = 10
sd = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pg.as_mdg(sd)

# Convert to a pygeon grid
pg.convert_from_pp(sd)
sd.compute_geometry()


With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
key = "flow"
bc_val = []
bc_ess = []
vector_source = []

RT0 = pg.RT0(key)
P0 = pg.PwConstants(key)

for sd, data in mdg.subdomains(return_data=True):
    # permeability tensor
    perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
    parameters = {
        "second_order_tensor": perm,
    }
    pp.initialize_data(sd, data, key, parameters)
    
    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    left_right = np.logical_or(sd.face_centers[0, :] == 0,  sd.face_centers[0, :] == 1)
    top_bottom = np.logical_or(sd.face_centers[1, :] == 0,  sd.face_centers[1, :] == 1)
    ess_p_dofs = np.zeros(P0.ndof(sd), dtype=bool)

    def p_bc(x): return x[1]
    bc_val.append(-RT0.assemble_nat_bc(sd, p_bc, top_bottom))
    bc_ess.append(np.hstack((left_right, ess_p_dofs)))
    
    def vector_source_fct(x): return np.array([0, -1, 0])
    mass = RT0.assemble_mass_matrix(sd)
    vector_source.append(mass @ RT0.interpolate(sd, vector_source_fct))

Once the data are assigned to the mixed-dimensional grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
M & -B^\top\\
B & 0
\end{array}
\right)
\left(
\begin{array}{c} 
q\\ 
p
\end{array}
\right)
=\left(
\begin{array}{c} 
p_{\partial} + g\\ 
0
\end{array}
\right)
$$<br>
where $p_{\partial}$ is the vector associated to the pressure boundary contions. To construct the saddle-point problem, we rely on the `scipy.sparse` function `bmat`. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [4]:
# construct the local matrices
mass = pg.face_mass(mdg)
div = pg.cell_mass(mdg) * pg.div(mdg)

# assemble the saddle point problem
spp = sps.bmat([[mass, -div.T], 
                [ div,  None]], format="csc")

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape

# assemble the right-hand side
rhs = np.zeros(dof_p + dof_q)
rhs[:dof_q] += np.hstack(bc_val) + np.hstack(vector_source)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (flux boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom. Once the problem is solved, we extract the two solutions $q$ and $p$.

In [5]:
# solve the problem
ls = pg.LinearSystem(spp, rhs)
ls.flag_ess_bc(np.hstack(bc_ess), np.zeros(dof_q + dof_p))
x = ls.solve()

# extract the variables
q = x[:dof_q]
p = x[-dof_p:]

Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [6]:
# post process variables
proj_q = RT0.eval_at_cell_centers(sd)
cell_q = (proj_q * q).reshape((3, -1), order="F")
cell_p = P0.eval_at_cell_centers(sd) * p

for _, data in mdg.subdomains(return_data=True):
    pp.set_solution_values("cell_q", cell_q, data, 0)
    pp.set_solution_values("cell_p", cell_p, data, 0)

save = pp.Exporter(mdg, "sol")
save.write_vtu(["cell_q", "cell_p"])

A representation of the computed solution is given below, where the cells are colored with $p$ and the arrows are the $q$. <br>
![](fig/darcy.png)